# Hurray basic usage

In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import hurraypy as hr
import numpy as np

Make sure a hurray server instance is running at localhost:2222.

In [9]:
hr.__version__

'0.0.3'

In [10]:
conn = hr.connect('localhost', '2222')

In [11]:
conn

<Connection (host=localhost, port=2222)>

In [12]:
db = conn.create_file("test.h5", overwrite=True)

In [13]:
db

<File (db=test.h5, path=/)>

In [14]:
db = conn.create_file("/home/rg/Desktop/test.h5", overwrite=True)

ServerError: (500, 'internal server error')

In [15]:
conn.File

hurraypy.client.Connection.__init__.<locals>._File

In [16]:
db = conn.File("test.h5")

In [17]:
db

<File (db=test.h5, path=/)>

In [18]:
print(db)

<File (db=test.h5, path=/)>


In [19]:
with conn.File("test.h5") as f:
    print(f)

<File (db=test.h5, path=/)>


In [20]:
db.attrs["bla"] = 123

In [21]:
db.attrs["bla"]

123

In [22]:
db

<File (db=test.h5, path=/)>

In [23]:
db.create_group("test")

<Group (db=test.h5, path=/test)>

In [24]:
db.keys()

('test',)

In [25]:
db["test"]

<Group (db=/home/rg/hurray_data/test.h5, path=/test)>

In [26]:
grp = db.create_group("mygrp/data")

In [27]:
db.keys()

('mygrp', 'test')

In [28]:
db["mygrp/data"]

<Group (db=/home/rg/hurray_data/test.h5, path=/mygrp/data)>

In [29]:
grp.path

'/mygrp/data'

In [30]:
db["/mygrp"]

<Group (db=/home/rg/hurray_data/test.h5, path=/mygrp)>

In [31]:
db["/mygrp"].attrs["foo"] = "bar"

In [32]:
db["/mygrp"]

<Group (db=/home/rg/hurray_data/test.h5, path=/mygrp)>

In [33]:
db["/mygrp"].keys()

('data',)

In [34]:
data = np.random.random((600, 400))

In [35]:
grp.create_dataset("myarray", data=data)

<Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>

In [36]:
dst = db["/mygrp/data/myarray"]

In [37]:
dst

<Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>

In [38]:
dst.attrs["unit"] = "celsius"

In [39]:
dst

<Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>

In [40]:
dst.shape

(600, 400)

In [41]:
dst.dtype

'float64'

In [42]:
dst[:]

array([[ 0.27587652,  0.97451559,  0.02597827, ...,  0.49549817,
         0.10602769,  0.54957183],
       [ 0.07280129,  0.57236653,  0.79046109, ...,  0.99726858,
         0.29241643,  0.81067177],
       [ 0.35915275,  0.3395208 ,  0.00605964, ...,  0.96693349,
         0.40918281,  0.23267074],
       ..., 
       [ 0.05391319,  0.70568114,  0.76275027, ...,  0.32788271,
         0.16523347,  0.66884406],
       [ 0.83034651,  0.54541909,  0.10944849, ...,  0.64470173,
         0.21839751,  0.54438394],
       [ 0.08265859,  0.59959771,  0.37144236, ...,  0.74178837,
         0.16370611,  0.24212855]])

In [43]:
dst[0, 0:3] = 999

In [44]:
dst[0, 0:4]

array([  9.99000000e+02,   9.99000000e+02,   9.99000000e+02,
         9.84715934e-02])

In [45]:
db

<File (db=test.h5, path=/)>

## Exploring file structures

Every group has a ``tree()`` method that displays sub groups and datasets as a tree.

In [46]:
db.require_group("/mygrp/data/somegroup")
db.require_group("/mygrp/datagrp2")
db.require_group("/mygrp/datagrp3")
db.require_group("/test/test2/test3")

<Group (db=test.h5, path=/test/test2/test3)>

In [47]:
db.tree()

── /
    ├─ mygrp
    │   ├─ data
    │   │   ├─ <Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>
    │   │   └─ somegroup
    │   ├─ datagrp2
    │   └─ datagrp3
    └─ test
        └─ test2
            └─ test3

In [48]:
db["/mygrp/data"].tree()

── data
    ├─ <Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>
    └─ somegroup

If you're not in a notebook or ipython console, ``tree()`` will give you a text based representation:

In [49]:
print(db.tree())

── /
    ├─ mygrp
    │   ├─ data
    │   │   ├─ <Dataset (600, 400) float64 (db=/home/rg/hurray_data/test.h5, path=/mygrp/data/myarray)>
    │   │   └─ somegroup
    │   ├─ datagrp2
    │   └─ datagrp3
    └─ test
        └─ test2
            └─ test3
